In [10]:
from sagemaker import get_execution_role
import time

role = get_execution_role()
sm = boto3.client('sagemaker')

In [14]:
model = 'cntkdemo-ecr-4-2018-01-21-13-26-49-604' # just an arbitrary name for the model instantiation

# model artifacts (see the "HowToPrepare" notebook)
model_data = 's3://sagemaker-us-east-2-012197625151/output/cntkdemo-ecr-4-2018-01-21-13-26-49-604/output/model.tar.gz'

image = '012197625151.dkr.ecr.us-east-2.amazonaws.com/cntkdemo-ecr-4'

In [16]:
%%time
create_model_response = sm.create_model(
    ModelName=model,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': model_data
    }
)

print(create_model_response['ModelArn'])

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:us-east-2:012197625151:model/cntkdemo-ecr-4-2018-01-21-13-26-49-604".

In [20]:
cntkdemo_endpoint_config = 'cntkdemo-poc-endpoint-config-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(cntkdemo_endpoint_config)
cntkdemo_endpoint = 'cntkdemo-poc-endpoint-' + time.strftime("%Y%m%d%H%M", time.gmtime())
print(cntkdemo_endpoint)

cntkdemo-poc-endpoint-config-2018-01-21-13-58-53
cntkdemo-poc-endpoint-201801211358


In [21]:
%%time
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=cntkdemo_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.m4.xlarge',
        'InitialInstanceCount': 1,
        'ModelName': model,
        'VariantName': 'cntkdemo'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint Config Arn: arn:aws:sagemaker:us-east-2:012197625151:endpoint-config/cntkdemo-poc-endpoint-config-2018-01-21-13-58-53
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 212 ms


In [22]:
# be patience this block can require 5 or more minutes for the execution
%%time
create_endpoint_response = sm.create_endpoint(
    EndpointName=cntkdemo_endpoint,
    EndpointConfigName=cntkdemo_endpoint_config)

print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=cntkdemo_endpoint)
status = resp['EndpointStatus']

print("Status: " + status)

sm.get_waiter('endpoint_in_service').wait(EndpointName=cntkdemo_endpoint)

resp = sm.describe_endpoint(EndpointName=cntkdemo_endpoint)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

if status != 'InService':
    raise Exception('Endpoint creation did not succeed')

arn:aws:sagemaker:us-east-2:012197625151:endpoint/cntkdemo-poc-endpoint-201801211358
Status: Creating
Arn: arn:aws:sagemaker:us-east-2:012197625151:endpoint/cntkdemo-poc-endpoint-201801211358
Status: InService
CPU times: user 112 ms, sys: 16 ms, total: 128 ms
Wall time: 13min 36s
